In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import urllib.request

url = 'http://ergast.com/downloads/f1db_csv.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zf = zipfile.ZipFile(filehandle, 'r')

results_df = pd.read_csv(zf.open('results.csv'))
drivers_df = pd.read_csv(zf.open('drivers.csv'))
lap_times_df = pd.read_csv(zf.open('lap_times.csv'))
pit_stops_df = pd.read_csv(zf.open('pit_stops.csv'))

In [2]:
raceId = lap_times_df.raceId.drop_duplicates().iloc[-1]
race_df = lap_times_df[(lap_times_df.raceId == raceId) & (lap_times_df.lap != 1)].copy()
pit_stop_df = pit_stops_df[pit_stops_df.raceId == raceId]

for driverId in pit_stop_df.driverId.drop_duplicates():
    for lap in pit_stop_df[pit_stop_df.driverId == driverId].lap:
        race_df.loc[(race_df.driverId == driverId) & (race_df.lap.isin([lap, lap + 1])),'milliseconds'] = np.nan

phase_data = []
        
for driverId in race_df.driverId.drop_duplicates():
    driver_lap_time_df = race_df[race_df.driverId == driverId].copy()
    driver_lap_time_df["delta_milliseconds"] = driver_lap_time_df.milliseconds - driver_lap_time_df.milliseconds.shift()
#     driver_lap_time_df["delta_milliseconds"] = driver_lap_time_df["delta_milliseconds"].fillna(0)
    driver_info = drivers_df[drivers_df["driverId"] == driverId]
    driver_name = driver_info["forename"] + ' ' + driver_info["surname"]
    phase_data.append([driver_name.iloc[0], driver_lap_time_df.delta_milliseconds.mean()])
    
pd.DataFrame(phase_data, columns = ["Driver name", "Phase earned"]).sort_values("Phase earned")

,Driver name,Phase earned
15,Esteban Ocon,-90.400000
11,Daniel Ricciardo,-87.866667
9,Lando Norris,-47.508772
6,Carlos Sainz,-38.228070
3,Valtteri Bottas,-33.181818
13,Kevin Magnussen,-30.166667
0,Lewis Hamilton,-17.758621
10,Sebastian Vettel,-2.850000
5,Alexander Albon,17.526316
7,Pierre Gasly,17.789474
